<a href="https://colab.research.google.com/github/valmirf/mineracao_textual/blob/main/CNN/05_CNN_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/valmirf/mineracao_textual.git

# Classificação de Textos com CNNs

## Carregar Dados

In [ ]:
import pandas as pd

df = data = pd.read_csv("mineracao_textual/Dados/ClothingReviews.csv")

# descartar classes minoritarias
major_classes = list(df['Class Name'].value_counts()[0:4].index)
majority      = df.loc[df['Class Name'].isin(major_classes),]
minority      = df.loc[~df['Class Name'].isin(major_classes),]

print('\n',5*'=', 'major classes: \n',    majority['Class Name'].value_counts()[0:4])
print('\n',5*'=', 'minority classes: \n', minority['Class Name'].value_counts()[4:])

majority.head()

## Codificar Labels e Separar treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)

_labels = majority['Class Name'].values.reshape((len(majority['Class Name']), 1))
X = majority['Review Text']
y = encoder.fit_transform(_labels)

sentences_train, sentences_test, y_train, y_test  = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
sentences_train[:4], y_train[:4]

In [ ]:
y_train.shape

## Tokenizando Texto

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)
X_train   = tokenizer.texts_to_sequences(sentences_train)
X_test    = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1 

print(vocab_size)
print(sentences_train.iloc[2])
print(X_train[2])

In [ ]:
#X_train

In [ ]:
#list(tokenizer.word_index)

## Sequence Padding

Um problema que temos é que cada sequência de texto tem na maioria dos casos diferentes comprimentos de palavras. Para corrigir isso, você pode usar `pad_sequence()` que simplesmente preenche a sequência de palavras com zeros. 

Por padrão, ele anexa zeros, mas queremos anexá-los. Normalmente, não importa se você acrescenta ou acrescenta zeros.

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :], y_train[0])

## Arquitetura da CNN

In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 300
num_classes = y_train.shape[1]

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

## Adicionando embeddings pré-treinados
É possível usarmos embeddings pré-treinados. A escolha é sempre relativa ao seu problema. Por exemplo, se você precisa resolver um problema de classificação de texto de cunho geral, pode pegar um Embeddings pré-treinado do Google, com milhões de textos. Porém, se quiser resolver um problema de classificação de sentimentos de review de livros, pode ser útil utilizar um embeddings mais próximo do seu problema, como por exemplo, um embeddings pré-treinado com informações e review de livros da Amazon. Nessa atividade, vamos utilizar....
### Word2Vec

In [ ]:
!wget https://gist.githubusercontent.com/bastings/4d1c346c68969b95f2c34cfbc00ba0a0/raw/76b4fefc9ef635a79d0d8002522543bc53ca2683/googlenews.word2vec.300d.txt

In [ ]:
!head -n 1 googlenews.word2vec.300d.txt | cut -c-50

### GLove

In [ ]:
!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt

In [ ]:
!head -n 1 glove.840B.300d.sst.txt | cut -c-50

## Teste: Word2Vec

In [ ]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix
    
embedding_matrix = create_embedding_matrix('googlenews.word2vec.300d.txt',
                      tokenizer.word_index, embedding_dim)

In [ ]:
embedding_matrix[0:3,:10]

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

## Teste: Glove

In [ ]:
embedding_matrix = create_embedding_matrix('glove.840B.300d.sst.txt',
                      tokenizer.word_index, embedding_dim)
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

## Métricas detalhadas

In [ ]:
from sklearn import metrics


y_true  = np.argmax(y_test, axis=1)
preds = (model.predict(X_test) > 0.5).astype("int32")[:, [1]]

print(metrics.classification_report(y_true,preds))

## Como melhorar?

- Hyperparameter tuning
- Adicionar mais filtros (convolution layers), por exemplo:

```
filter_sizes = [3,4,5]
num_filters = 512

...

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)

...
```